In [46]:
# Import libraries for data manipulation and ETL pipeline
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
from pandarallel import pandarallel
pandarallel.initialize(verbose=0)
from tqdm import tqdm

In [47]:
# Establish API connection
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

cid = '3fda75b7146a4769b207ee44017b3abe'
secret = '2a755cb04a18406b9394dbef2f8069dd'

client_credentials_manager = SpotifyClientCredentials(client_id=cid, client_secret=secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [48]:
# Connect to SQL Server
import sqlalchemy as sql
from sqlalchemy import text
import pyodbc
server = 'LAPTOP-V3754MEK' 
database = 'Spotify'

engine = sql.create_engine('mssql+pyodbc://' + server + '/' + database + '?trusted_connection=yes&driver=ODBC+Driver+13+for+SQL+Server')
conn = engine.connect()
results = engine.execute("SELECT track_uri2 FROM dim_tracks ORDER BY track_uri2")
results = pd.Series(results)
results.head()

0    (spotify:track:0002yNGLtYSYtc0X6ZnFvp)
1    (spotify:track:00039MgrmLoIzSpuYKurn9)
2    (spotify:track:0003Z98F6hUq7XxqSRM87H)
3    (spotify:track:0004ExljAge0P5XWn1LXmW)
4    (spotify:track:0005rgjsSeVLp1cze57jIN)
dtype: object

In [49]:
# Extract tracks from API
output_df = pd.DataFrame()
for row in tqdm(results[920001:930001]):
      output = pd.DataFrame(sp.audio_features(row))
      output_df = pd.concat([output_df, output])
output_df.head()

100%|██████████| 10000/10000 [15:58<00:00, 10.43it/s]


,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,0.295,0.4780,11,-7.997,1,0.0755,0.0232,0.000000,0.294,0.1800,161.042,audio_features,39zAOQvdaBUG6Ws9FeiLyi,spotify:track:39zAOQvdaBUG6Ws9FeiLyi,https://api.spotify.com/v1/tracks/39zAOQvdaBUG...,https://api.spotify.com/v1/audio-analysis/39zA...,386040,4
0,0.321,0.0865,5,-16.822,1,0.0398,0.8270,0.000021,0.249,0.1170,91.706,audio_features,39zAUogcvW9wHZTQyiwLxO,spotify:track:39zAUogcvW9wHZTQyiwLxO,https://api.spotify.com/v1/tracks/39zAUogcvW9w...,https://api.spotify.com/v1/audio-analysis/39zA...,116613,3
0,0.668,0.6520,9,-7.088,0,0.0970,0.2130,0.001610,0.131,0.5770,103.110,audio_features,39zB6SRQQjMTi2B076tfAS,spotify:track:39zB6SRQQjMTi2B076tfAS,https://api.spotify.com/v1/tracks/39zB6SRQQjMT...,https://api.spotify.com/v1/audio-analysis/39zB...,213413,4
0,0.283,0.2130,8,-16.570,1,0.0322,0.7140,0.869000,0.138,0.3300,112.683,audio_features,39ZBweQvRFVxKPz163Kgyp,spotify:track:39ZBweQvRFVxKPz163Kgyp,https://api.spotify.com/v1/tracks/39ZBweQvRFVx...,https://api.spotify.com/v1/audio-analysis/39ZB...,272536,1
0,0.286,0.2310,2,-17.363,1,0.0430,0.9050,0.066700,0.171,0.0644,130.413,audio_features,39ZcDNZUE4IacpvhxbBUub,spotify:track:39ZcDNZUE4IacpvhxbBUub,https://api.spotify.com/v1/tracks/39ZcDNZUE4Ia...,https://api.spotify.com/v1/audio-analysis/39Zc...,1520493,4


In [50]:
# Load data into SQL Server
output_df = output_df[['uri', 'danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'duration_ms', 'time_signature']]
output_df = output_df.rename(columns={'key':'music_key', 'mode':'modal'})
output_df.to_sql("audio_features2", engine, if_exists='append', index=False)
print(f'{len(output_df)} rows imported to database.')
new_query = engine.execute('SELECT COUNT(*) FROM audio_features2')
new_query = pd.Series(new_query)
print(f'App contains {new_query[0]} songs.')

10000 rows imported to database.
App contains (920001,) songs.
